# Machine Learning Diabetes Classification

## Read csv and perform basic data cleaning

In [1]:
# Install zipfile36 if you haven't already
# !pip install zipfile36

In [1]:
# Import our dependencies
import pandas as pd
import numpy as np
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

# SQL
from sqlalchemy import create_engine
import sqlite3 as sql

# Machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score
import tensorflow as tf

In [2]:
# Create dataframe
z = urlopen('https://archive.ics.uci.edu/ml/machine-learning-databases/00296/dataset_diabetes.zip')
myzip = ZipFile(BytesIO(z.read())).extract('dataset_diabetes/diabetic_data.csv')
df = pd.read_csv(myzip)
df.head(5)
target = ['A1Cresult']

In [3]:
# Drop the non-beneficial ID columns, 'encounter_id' and 'patient_nbr'
df = df.drop(['patient_nbr'],1)

# Drop mostly empty columns, 'weight', 'payer_code', 'max_glu_serum', and 'medical_specialty'
df = df.drop(['weight', 'payer_code', 'max_glu_serum', 'medical_specialty', ],1)

# Replace '?' values to nulls
df.replace({'?': np.nan}, inplace=True)

# Replace 'None' values to nulls
df.replace({'None': np.nan}, inplace=True)

# Drop the null rows
df = df.dropna()

/Users/juliaheuer/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  
/Users/juliaheuer/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """


In [4]:
# Convert the target column values to normal and high based on their values
x = {'Norm': 'low'}   
df = df.replace(x)
x = dict.fromkeys(['>7', '>8'], 'high')    
df = df.replace(x)
df.reset_index(inplace=True, drop=True)

In [5]:
# Determine the number of unique values in each column.
df.nunique()

encounter_id                16193
race                            5
gender                          2
age                            10
admission_type_id               8
discharge_disposition_id       21
admission_source_id            15
time_in_hospital               14
num_lab_procedures            114
num_procedures                  7
num_medications                67
number_outpatient              24
number_emergency               19
number_inpatient               18
diag_1                        490
diag_2                        486
diag_3                        539
number_diagnoses               12
A1Cresult                       2
metformin                       4
repaglinide                     4
nateglinide                     4
chlorpropamide                  2
glimepiride                     4
acetohexamide                   1
glipizide                       4
glyburide                       4
tolbutamide                     2
pioglitazone                    4
rosiglitazone 

In [6]:
# Drop columns with only 1 value
df = df.drop(['acetohexamide', 'troglitazone', 'examide', 'citoglipton','glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone'],1)
df.nunique()

/Users/juliaheuer/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


encounter_id                16193
race                            5
gender                          2
age                            10
admission_type_id               8
discharge_disposition_id       21
admission_source_id            15
time_in_hospital               14
num_lab_procedures            114
num_procedures                  7
num_medications                67
number_outpatient              24
number_emergency               19
number_inpatient               18
diag_1                        490
diag_2                        486
diag_3                        539
number_diagnoses               12
A1Cresult                       2
metformin                       4
repaglinide                     4
nateglinide                     4
chlorpropamide                  2
glimepiride                     4
glipizide                       4
glyburide                       4
tolbutamide                     2
pioglitazone                    4
rosiglitazone                   4
acarbose      

In [7]:
#Info on the data frame data types, .non_null, etc. 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16193 entries, 0 to 16192
Data columns (total 38 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   encounter_id              16193 non-null  int64 
 1   race                      16193 non-null  object
 2   gender                    16193 non-null  object
 3   age                       16193 non-null  object
 4   admission_type_id         16193 non-null  int64 
 5   discharge_disposition_id  16193 non-null  int64 
 6   admission_source_id       16193 non-null  int64 
 7   time_in_hospital          16193 non-null  int64 
 8   num_lab_procedures        16193 non-null  int64 
 9   num_procedures            16193 non-null  int64 
 10  num_medications           16193 non-null  int64 
 11  number_outpatient         16193 non-null  int64 
 12  number_emergency          16193 non-null  int64 
 13  number_inpatient          16193 non-null  int64 
 14  diag_1                

In [8]:
#Looking at dataframe 
df.head()

,encounter_id,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,...,rosiglitazone,acarbose,miglitol,tolazamide,insulin,glyburide-metformin,glipizide-metformin,change,diabetesMed,readmitted
0,236316,Caucasian,Male,[80-90),1,3,7,6,64,3,...,No,No,No,No,No,No,No,Ch,Yes,NO
1,955884,Caucasian,Female,[70-80),1,3,7,5,34,0,...,No,No,No,No,Up,No,No,Ch,Yes,>30
2,1257282,Other,Female,[50-60),1,1,7,2,53,0,...,No,No,No,No,Up,No,No,Ch,Yes,NO
3,1270524,Caucasian,Male,[60-70),1,2,7,1,59,0,...,No,No,No,No,Steady,No,No,No,Yes,NO
4,1455252,Caucasian,Female,[80-90),1,1,7,3,34,0,...,No,No,No,No,No,No,No,No,No,>30


In [9]:
# Generate our categorical variable list
df_cat = df.dtypes[df.dtypes == "object"].index.tolist()
df_cat

['race',
 'gender',
 'age',
 'diag_1',
 'diag_2',
 'diag_3',
 'A1Cresult',
 'metformin',
 'repaglinide',
 'nateglinide',
 'chlorpropamide',
 'glimepiride',
 'glipizide',
 'glyburide',
 'tolbutamide',
 'pioglitazone',
 'rosiglitazone',
 'acarbose',
 'miglitol',
 'tolazamide',
 'insulin',
 'glyburide-metformin',
 'glipizide-metformin',
 'change',
 'diabetesMed',
 'readmitted']

In [10]:
# Check the number of unique values in each column
df[df_cat].nunique()

race                     5
gender                   2
age                     10
diag_1                 490
diag_2                 486
diag_3                 539
A1Cresult                2
metformin                4
repaglinide              4
nateglinide              4
chlorpropamide           2
glimepiride              4
glipizide                4
glyburide                4
tolbutamide              2
pioglitazone             4
rosiglitazone            4
acarbose                 4
miglitol                 4
tolazamide               3
insulin                  4
glyburide-metformin      4
glipizide-metformin      2
change                   2
diabetesMed              2
readmitted               3
dtype: int64

In [11]:
# Check the unique value counts to see if binning is required
df.diag_1.value_counts()

428    1144
414     976
786     853
410     806
486     531
       ... 
237       1
617       1
356       1
989       1
893       1
Name: diag_1, Length: 490, dtype: int64

In [12]:
# Check the unique value counts to see if binning is required
df.diag_2.value_counts()

276       1280
428        938
250        753
427        739
250.02     677
          ... 
316          1
E858         1
980          1
725          1
110          1
Name: diag_2, Length: 486, dtype: int64

In [13]:
# Check the unique value counts to see if binning is required
df.diag_3.value_counts()

250    1557
401    1229
276    1057
428     665
414     550
       ... 
550       1
V55       1
314       1
579       1
825       1
Name: diag_3, Length: 539, dtype: int64

In [14]:
# Create SQLite engine using SQLAlechmey
engine = create_engine('sqlite:///diabetes_data.db', echo=False)
conn = engine.connect()
df.to_sql('diabetes', conn, index=False, if_exists='replace')

In [15]:
# Check dataset
pd.read_sql('SELECT * FROM diabetes LIMIT 10', conn)

,encounter_id,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,...,rosiglitazone,acarbose,miglitol,tolazamide,insulin,glyburide-metformin,glipizide-metformin,change,diabetesMed,readmitted
0,236316,Caucasian,Male,[80-90),1,3,7,6,64,3,...,No,No,No,No,No,No,No,Ch,Yes,NO
1,955884,Caucasian,Female,[70-80),1,3,7,5,34,0,...,No,No,No,No,Up,No,No,Ch,Yes,>30
2,1257282,Other,Female,[50-60),1,1,7,2,53,0,...,No,No,No,No,Up,No,No,Ch,Yes,NO
3,1270524,Caucasian,Male,[60-70),1,2,7,1,59,0,...,No,No,No,No,Steady,No,No,No,Yes,NO
4,1455252,Caucasian,Female,[80-90),1,1,7,3,34,0,...,No,No,No,No,No,No,No,No,No,>30
5,1810752,Caucasian,Male,[70-80),1,3,7,14,78,1,...,No,No,No,No,Up,No,No,Ch,Yes,<30
6,1881372,Caucasian,Male,[60-70),1,2,7,4,65,2,...,No,No,No,No,Steady,No,No,No,Yes,<30
7,1968528,Caucasian,Female,[70-80),6,25,1,10,56,2,...,No,No,No,No,Down,No,No,Ch,Yes,>30
8,2092362,Caucasian,Female,[70-80),6,25,7,11,88,1,...,No,No,No,No,Down,No,No,Ch,Yes,>30
9,2095932,AfricanAmerican,Female,[30-40),6,25,7,8,62,0,...,No,No,No,No,Steady,No,No,Ch,Yes,>30


In [16]:
# Create cleanup table
engine.execute('CREATE TABLE "cleaned_columns" ('
               'id BIGINT NOT NULL,'
               'diag_1c VARCHAR, '
               'diag_2c VARCHAR, '
               'diag_3c VARCHAR, '
               'PRIMARY KEY (id));')

OperationalError: (sqlite3.OperationalError) table "cleaned_columns" already exists
[SQL: CREATE TABLE "cleaned_columns" (id BIGINT NOT NULL,diag_1c VARCHAR, diag_2c VARCHAR, diag_3c VARCHAR, PRIMARY KEY (id));]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [17]:
# Insert columns into new table to clean
engine.execute('INSERT INTO "cleaned_columns" '
               'SELECT encounter_id AS id, '
               'CASE '
                   'WHEN (diag_1 BETWEEN 390 AND 459) OR (diag_1 = 785) THEN "circulatory" '
                   'WHEN (diag_1 BETWEEN 460 AND 519) OR (diag_1 = 786) THEN "respiratory" '
                   'WHEN (diag_1 BETWEEN 520 AND 579) OR (diag_1 = 787) THEN "digestive" '
                   'WHEN (diag_1 BETWEEN 250 AND 251) THEN "diabetes" '
                   'WHEN (diag_1 BETWEEN 800 AND 999) THEN "injury" '
                   'WHEN (diag_1 BETWEEN 710 AND 739) THEN "musculoskeletal" '
                   'WHEN (diag_1 BETWEEN 580 AND 629) OR (diag_1 = 788) THEN "genitourinary" '
                   'WHEN (diag_1 BETWEEN 140 AND 239) THEN "neoplasms" '
                   'ELSE "other" '
               'END AS diag_1c, '
               'CASE '
                   'WHEN (diag_2 BETWEEN 390 AND 459) OR (diag_2 = 785) THEN "circulatory" '
                   'WHEN (diag_2 BETWEEN 460 AND 519) OR (diag_2 = 786) THEN "respiratory" '
                   'WHEN (diag_2 BETWEEN 520 AND 579) OR (diag_2 = 787) THEN "digestive" '
                   'WHEN (diag_2 BETWEEN 250 AND 251) THEN "diabetes" '
                   'WHEN (diag_2 BETWEEN 800 AND 999) THEN "injury" '
                   'WHEN (diag_2 BETWEEN 710 AND 739) THEN "musculoskeletal" '
                   'WHEN (diag_2 BETWEEN 580 AND 629) OR (diag_2 = 788) THEN "genitourinary" '
                   'WHEN (diag_2 BETWEEN 140 AND 239) THEN "neoplasms" '
                   'ELSE "other" '
               'END AS diag_2c, '
               'CASE '
                   'WHEN (diag_3 BETWEEN 390 AND 459) OR (diag_3 = 785) THEN "circulatory" '
                   'WHEN (diag_3 BETWEEN 460 AND 519) OR (diag_3 = 786) THEN "respiratory" '
                   'WHEN (diag_3 BETWEEN 520 AND 579) OR (diag_3 = 787) THEN "digestive" '
                   'WHEN (diag_3 BETWEEN 250 AND 251) THEN "diabetes" '
                   'WHEN (diag_3 BETWEEN 800 AND 999) THEN "injury" '
                   'WHEN (diag_3 BETWEEN 710 AND 739) THEN "musculoskeletal" '
                   'WHEN (diag_3 BETWEEN 580 AND 629) OR (diag_3 = 788) THEN "genitourinary" '
                   'WHEN (diag_3 BETWEEN 140 AND 239) THEN "neoplasms" '
                   'ELSE "other" '
               'END AS diag_3c '
               'FROM diabetes;')

IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: cleaned_columns.id
[SQL: INSERT INTO "cleaned_columns" SELECT encounter_id AS id, CASE WHEN (diag_1 BETWEEN 390 AND 459) OR (diag_1 = 785) THEN "circulatory" WHEN (diag_1 BETWEEN 460 AND 519) OR (diag_1 = 786) THEN "respiratory" WHEN (diag_1 BETWEEN 520 AND 579) OR (diag_1 = 787) THEN "digestive" WHEN (diag_1 BETWEEN 250 AND 251) THEN "diabetes" WHEN (diag_1 BETWEEN 800 AND 999) THEN "injury" WHEN (diag_1 BETWEEN 710 AND 739) THEN "musculoskeletal" WHEN (diag_1 BETWEEN 580 AND 629) OR (diag_1 = 788) THEN "genitourinary" WHEN (diag_1 BETWEEN 140 AND 239) THEN "neoplasms" ELSE "other" END AS diag_1c, CASE WHEN (diag_2 BETWEEN 390 AND 459) OR (diag_2 = 785) THEN "circulatory" WHEN (diag_2 BETWEEN 460 AND 519) OR (diag_2 = 786) THEN "respiratory" WHEN (diag_2 BETWEEN 520 AND 579) OR (diag_2 = 787) THEN "digestive" WHEN (diag_2 BETWEEN 250 AND 251) THEN "diabetes" WHEN (diag_2 BETWEEN 800 AND 999) THEN "injury" WHEN (diag_2 

In [18]:
# Check data
pd.read_sql('SELECT * FROM cleaned_columns LIMIT 10', conn)

,id,diag_1c,diag_2c,diag_3c
0,236316,circulatory,circulatory,circulatory
1,955884,circulatory,circulatory,other
2,1257282,genitourinary,diabetes,circulatory
3,1270524,circulatory,circulatory,respiratory
4,1455252,circulatory,circulatory,circulatory
5,1810752,circulatory,diabetes,diabetes
6,1881372,circulatory,circulatory,respiratory
7,1968528,circulatory,circulatory,diabetes
8,2092362,diabetes,other,genitourinary
9,2095932,diabetes,circulatory,other


In [21]:
# Join diabetes and cleaned_columns tables
diabetes_df = pd.read_sql('SELECT * FROM diabetes JOIN cleaned_columns ON diabetes.encounter_id = cleaned_columns.id', conn)
diabetes_df 

,encounter_id,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,...,insulin,glyburide-metformin,glipizide-metformin,change,diabetesMed,readmitted,id,diag_1c,diag_2c,diag_3c
0,236316,Caucasian,Male,[80-90),1,3,7,6,64,3,...,No,No,No,Ch,Yes,NO,236316,circulatory,circulatory,circulatory
1,955884,Caucasian,Female,[70-80),1,3,7,5,34,0,...,Up,No,No,Ch,Yes,>30,955884,circulatory,circulatory,other
2,1257282,Other,Female,[50-60),1,1,7,2,53,0,...,Up,No,No,Ch,Yes,NO,1257282,genitourinary,diabetes,circulatory
3,1270524,Caucasian,Male,[60-70),1,2,7,1,59,0,...,Steady,No,No,No,Yes,NO,1270524,circulatory,circulatory,respiratory
4,1455252,Caucasian,Female,[80-90),1,1,7,3,34,0,...,No,No,No,No,No,>30,1455252,circulatory,circulatory,circulatory
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16188,443816024,Caucasian,Female,[70-80),3,6,1,3,27,1,...,Steady,No,No,Ch,Yes,NO,443816024,musculoskeletal,circulatory,diabetes
16189,443835140,Caucasian,Male,[70-80),3,6,1,13,77,6,...,Up,No,No,Ch,Yes,NO,443835140,circulatory,circulatory,respiratory
16190,443842016,Caucasian,Female,[70-80),1,1,7,9,50,2,...,Steady,No,No,Ch,Yes,>30,443842016,digestive,digestive,diabetes
16191,443842022,Other,Female,[40-50),1,1,7,14,73,6,...,Up,No,No,Ch,Yes,>30,443842022,genitourinary,genitourinary,respiratory


In [22]:
#Check Datatypes
diabetes_df.dtypes

encounter_id                 int64
race                        object
gender                      object
age                         object
admission_type_id            int64
discharge_disposition_id     int64
admission_source_id          int64
time_in_hospital             int64
num_lab_procedures           int64
num_procedures               int64
num_medications              int64
number_outpatient            int64
number_emergency             int64
number_inpatient             int64
diag_1                      object
diag_2                      object
diag_3                      object
number_diagnoses             int64
A1Cresult                   object
metformin                   object
repaglinide                 object
nateglinide                 object
chlorpropamide              object
glimepiride                 object
glipizide                   object
glyburide                   object
tolbutamide                 object
pioglitazone                object
rosiglitazone       

In [23]:
diabetes_df

,encounter_id,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,...,insulin,glyburide-metformin,glipizide-metformin,change,diabetesMed,readmitted,id,diag_1c,diag_2c,diag_3c
0,236316,Caucasian,Male,[80-90),1,3,7,6,64,3,...,No,No,No,Ch,Yes,NO,236316,circulatory,circulatory,circulatory
1,955884,Caucasian,Female,[70-80),1,3,7,5,34,0,...,Up,No,No,Ch,Yes,>30,955884,circulatory,circulatory,other
2,1257282,Other,Female,[50-60),1,1,7,2,53,0,...,Up,No,No,Ch,Yes,NO,1257282,genitourinary,diabetes,circulatory
3,1270524,Caucasian,Male,[60-70),1,2,7,1,59,0,...,Steady,No,No,No,Yes,NO,1270524,circulatory,circulatory,respiratory
4,1455252,Caucasian,Female,[80-90),1,1,7,3,34,0,...,No,No,No,No,No,>30,1455252,circulatory,circulatory,circulatory
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16188,443816024,Caucasian,Female,[70-80),3,6,1,3,27,1,...,Steady,No,No,Ch,Yes,NO,443816024,musculoskeletal,circulatory,diabetes
16189,443835140,Caucasian,Male,[70-80),3,6,1,13,77,6,...,Up,No,No,Ch,Yes,NO,443835140,circulatory,circulatory,respiratory
16190,443842016,Caucasian,Female,[70-80),1,1,7,9,50,2,...,Steady,No,No,Ch,Yes,>30,443842016,digestive,digestive,diabetes
16191,443842022,Other,Female,[40-50),1,1,7,14,73,6,...,Up,No,No,Ch,Yes,>30,443842022,genitourinary,genitourinary,respiratory


In [24]:
# Check the unique value counts to see if binning is required
diabetes_df.A1Cresult.value_counts()


high    11339
low      4854
Name: A1Cresult, dtype: int64

In [ ]:
#taking target out of df 
# object_db_df = 

In [18]:
# Chceck to see if we need to bin any of the groups. 

# Bin the values based on the medical codes 

In [ ]:
#Drop 'Change' variable. 

In [25]:
# Generate our categorical variable list
diabetes_df_cat = diabetes_df.dtypes[diabetes_df.dtypes == "object"].index.tolist()
diabetes_df_cat

['race',
 'gender',
 'age',
 'diag_1',
 'diag_2',
 'diag_3',
 'A1Cresult',
 'metformin',
 'repaglinide',
 'nateglinide',
 'chlorpropamide',
 'glimepiride',
 'glipizide',
 'glyburide',
 'tolbutamide',
 'pioglitazone',
 'rosiglitazone',
 'acarbose',
 'miglitol',
 'tolazamide',
 'insulin',
 'glyburide-metformin',
 'glipizide-metformin',
 'change',
 'diabetesMed',
 'readmitted',
 'diag_1c',
 'diag_2c',
 'diag_3c']

In [37]:
# Drop the non-beneficial ID and diag columns, 'encounter_id', 'id', 'diag_1', 'diag_2', and 'diag_3'
diabetes_2_df = diabetes_df.drop(['encounter_id', 'id', 'diag_1', 'diag_2', 'diag_3'],1)

/Users/juliaheuer/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [38]:
diabetes_2_df 

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,...,tolazamide,insulin,glyburide-metformin,glipizide-metformin,change,diabetesMed,readmitted,diag_1c,diag_2c,diag_3c
0,Caucasian,Male,[80-90),1,3,7,6,64,3,18,...,No,No,No,No,Ch,Yes,NO,circulatory,circulatory,circulatory
1,Caucasian,Female,[70-80),1,3,7,5,34,0,17,...,No,Up,No,No,Ch,Yes,>30,circulatory,circulatory,other
2,Other,Female,[50-60),1,1,7,2,53,0,6,...,No,Up,No,No,Ch,Yes,NO,genitourinary,diabetes,circulatory
3,Caucasian,Male,[60-70),1,2,7,1,59,0,12,...,No,Steady,No,No,No,Yes,NO,circulatory,circulatory,respiratory
4,Caucasian,Female,[80-90),1,1,7,3,34,0,11,...,No,No,No,No,No,No,>30,circulatory,circulatory,circulatory
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16188,Caucasian,Female,[70-80),3,6,1,3,27,1,29,...,No,Steady,No,No,Ch,Yes,NO,musculoskeletal,circulatory,diabetes
16189,Caucasian,Male,[70-80),3,6,1,13,77,6,65,...,No,Up,No,No,Ch,Yes,NO,circulatory,circulatory,respiratory
16190,Caucasian,Female,[70-80),1,1,7,9,50,2,33,...,No,Steady,No,No,Ch,Yes,>30,digestive,digestive,diabetes
16191,Other,Female,[40-50),1,1,7,14,73,6,26,...,No,Up,No,No,Ch,Yes,>30,genitourinary,genitourinary,respiratory


In [39]:
# Check data
diabetes_2_df.dtypes

race                        object
gender                      object
age                         object
admission_type_id            int64
discharge_disposition_id     int64
admission_source_id          int64
time_in_hospital             int64
num_lab_procedures           int64
num_procedures               int64
num_medications              int64
number_outpatient            int64
number_emergency             int64
number_inpatient             int64
number_diagnoses             int64
A1Cresult                   object
metformin                   object
repaglinide                 object
nateglinide                 object
chlorpropamide              object
glimepiride                 object
glipizide                   object
glyburide                   object
tolbutamide                 object
pioglitazone                object
rosiglitazone               object
acarbose                    object
miglitol                    object
tolazamide                  object
insulin             

In [40]:
# # Generate our categorical variable list
diabetes_2_df_cat = diabetes_2_df.dtypes[diabetes_2_df.dtypes == "object"].index.tolist()
diabetes_2_df_cat


['race',
 'gender',
 'age',
 'A1Cresult',
 'metformin',
 'repaglinide',
 'nateglinide',
 'chlorpropamide',
 'glimepiride',
 'glipizide',
 'glyburide',
 'tolbutamide',
 'pioglitazone',
 'rosiglitazone',
 'acarbose',
 'miglitol',
 'tolazamide',
 'insulin',
 'glyburide-metformin',
 'glipizide-metformin',
 'change',
 'diabetesMed',
 'readmitted',
 'diag_1c',
 'diag_2c',
 'diag_3c']

In [48]:
diabetes_2_df_cat

['race',
 'gender',
 'age',
 'A1Cresult',
 'metformin',
 'repaglinide',
 'nateglinide',
 'chlorpropamide',
 'glimepiride',
 'glipizide',
 'glyburide',
 'tolbutamide',
 'pioglitazone',
 'rosiglitazone',
 'acarbose',
 'miglitol',
 'tolazamide',
 'insulin',
 'glyburide-metformin',
 'glipizide-metformin',
 'change',
 'diabetesMed',
 'readmitted',
 'diag_1c',
 'diag_2c',
 'diag_3c']

In [95]:
# #create datframe without A1C Results
# db_2_df = diabetes_df.drop(columns = "A1Cresult")
# db_2_df

In [96]:
# db_2_df.dtypes 


In [97]:
# # Generate our categorical variable list
# db_df_cat = db_2_df.dtypes[db_2_df.dtypes == "object"].index.tolist()
# db_df_cat


In [46]:
# Error somewhere is this frame. Missing values 

#OneHot encode the categorical data. 
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(diabetes_2_df[diabetes_2_df_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names_out(diabetes_2_df_cat)
encode_df.head()

,race_AfricanAmerican,race_Asian,race_Caucasian,race_Hispanic,race_Other,gender_Female,gender_Male,age_[0-10),age_[10-20),age_[20-30),...,diag_2c_respiratory,diag_3c_circulatory,diag_3c_diabetes,diag_3c_digestive,diag_3c_genitourinary,diag_3c_injury,diag_3c_musculoskeletal,diag_3c_neoplasms,diag_3c_other,diag_3c_respiratory
0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
encode_df.info

<bound method DataFrame.info of        race_AfricanAmerican  race_Asian  race_Caucasian  race_Hispanic  \
0                       0.0         0.0             1.0            0.0   
1                       0.0         0.0             1.0            0.0   
2                       0.0         0.0             0.0            0.0   
3                       0.0         0.0             1.0            0.0   
4                       0.0         0.0             1.0            0.0   
...                     ...         ...             ...            ...   
16188                   0.0         0.0             1.0            0.0   
16189                   0.0         0.0             1.0            0.0   
16190                   0.0         0.0             1.0            0.0   
16191                   0.0         0.0             0.0            0.0   
16192                   1.0         0.0             0.0            0.0   

       race_Other  gender_Female  gender_Male  age_[0-10)  age_[10-20)  \
0    

In [43]:
encode_df.head(20)

,race_AfricanAmerican,race_Asian,race_Caucasian,race_Hispanic,race_Other,gender_Female,gender_Male,age_[0-10),age_[10-20),age_[20-30),...,diag_2c_respiratory,diag_3c_circulatory,diag_3c_diabetes,diag_3c_digestive,diag_3c_genitourinary,diag_3c_injury,diag_3c_musculoskeletal,diag_3c_neoplasms,diag_3c_other,diag_3c_respiratory
0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [65]:
for col_name in encode_df.columns: 
    print(col_name)

race_AfricanAmerican
race_Asian
race_Caucasian
race_Hispanic
race_Other
gender_Female
gender_Male
age_[0-10)
age_[10-20)
age_[20-30)
age_[30-40)
age_[40-50)
age_[50-60)
age_[60-70)
age_[70-80)
age_[80-90)
age_[90-100)
A1Cresult_high
A1Cresult_low
metformin_Down
metformin_No
metformin_Steady
metformin_Up
repaglinide_Down
repaglinide_No
repaglinide_Steady
repaglinide_Up
nateglinide_Down
nateglinide_No
nateglinide_Steady
nateglinide_Up
chlorpropamide_No
chlorpropamide_Steady
glimepiride_Down
glimepiride_No
glimepiride_Steady
glimepiride_Up
glipizide_Down
glipizide_No
glipizide_Steady
glipizide_Up
glyburide_Down
glyburide_No
glyburide_Steady
glyburide_Up
tolbutamide_No
tolbutamide_Steady
pioglitazone_Down
pioglitazone_No
pioglitazone_Steady
pioglitazone_Up
rosiglitazone_Down
rosiglitazone_No
rosiglitazone_Steady
rosiglitazone_Up
acarbose_Down
acarbose_No
acarbose_Steady
acarbose_Up
miglitol_Down
miglitol_No
miglitol_Steady
miglitol_Up
tolazamide_No
tolazamide_Steady
tolazamide_Up
insulin_D

In [74]:
# # Merge one-hot encoded features and drop the originals
diabetes_2_df = diabetes_2_df.merge(encode_df,left_index=True, right_index=True)
diabetes_2_df = diabetes_2_df.drop(diabetes_2_df_cat,1)
diabetes_2_df.head()

/Users/juliaheuer/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,...,diag_2c_respiratory,diag_3c_circulatory,diag_3c_diabetes,diag_3c_digestive,diag_3c_genitourinary,diag_3c_injury,diag_3c_musculoskeletal,diag_3c_neoplasms,diag_3c_other,diag_3c_respiratory
0,1,3,7,6,64,3,18,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,3,7,5,34,0,17,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,1,7,2,53,0,6,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,2,7,1,59,0,12,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1,1,7,3,34,0,11,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [76]:
for col_name in diabetes_2_df.columns: 
    print(col_name)

admission_type_id
discharge_disposition_id
admission_source_id
time_in_hospital
num_lab_procedures
num_procedures
num_medications
number_outpatient
number_emergency
number_inpatient
number_diagnoses
race_AfricanAmerican
race_Asian
race_Caucasian
race_Hispanic
race_Other
gender_Female
gender_Male
age_[0-10)
age_[10-20)
age_[20-30)
age_[30-40)
age_[40-50)
age_[50-60)
age_[60-70)
age_[70-80)
age_[80-90)
age_[90-100)
A1Cresult_high
A1Cresult_low
metformin_Down
metformin_No
metformin_Steady
metformin_Up
repaglinide_Down
repaglinide_No
repaglinide_Steady
repaglinide_Up
nateglinide_Down
nateglinide_No
nateglinide_Steady
nateglinide_Up
chlorpropamide_No
chlorpropamide_Steady
glimepiride_Down
glimepiride_No
glimepiride_Steady
glimepiride_Up
glipizide_Down
glipizide_No
glipizide_Steady
glipizide_Up
glyburide_Down
glyburide_No
glyburide_Steady
glyburide_Up
tolbutamide_No
tolbutamide_Steady
pioglitazone_Down
pioglitazone_No
pioglitazone_Steady
pioglitazone_Up
rosiglitazone_Down
rosiglitazone_No
ro

In [88]:
# Remove loan status target from features data
y = diabetes_2_df['A1Cresult_high']
X = diabetes_2_df.drop(columns=['A1Cresult_high', 'A1Cresult_low'])
# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [89]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Random forest predictive accuracy: 0.705


In [90]:
# # Define the model - deep neural net
# number_input_features = len(X_train[0])
# hidden_nodes_layer1 = 8
# hidden_nodes_layer2 = 5

# nn = tf.keras.models.Sequential()

# # First hidden layer
# nn.add(
#     tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
# )

# # Second hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# # Output layer
# nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# # Check the structure of the model
# nn.summary()

In [94]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 24
hidden_nodes_layer2 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2022-02-08 16:20:31.566622: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/50
380/380 [==============================] - 1s 822us/step - loss: 0.6511 - accuracy: 0.6449
Epoch 2/50
380/380 [==============================] - 0s 793us/step - loss: 0.5773 - accuracy: 0.7037
Epoch 3/50
380/380 [==============================] - 0s 790us/step - loss: 0.5648 - accuracy: 0.7099
Epoch 4/50
380/380 [==============================] - 0s 798us/step - loss: 0.5579 - accuracy: 0.7171
Epoch 5/50
380/380 [==============================] - 0s 797us/step - loss: 0.5521 - accuracy: 0.7202
Epoch 6/50
380/380 [==============================] - 0s 783us/step - loss: 0.5471 - accuracy: 0.7238
Epoch 7/50
380/380 [==============================] - 0s 832us/step - loss: 0.5418 - accuracy: 0.7270
Epoch 8/50
380/380 [==============================] - 0s 789us/step - loss: 0.5380 - accuracy: 0.7298
Epoch 9/50
380/380 [==============================] - 0s 797us/step - loss: 0.5329 - accuracy: 0.7336
Epoch 10/50
380/380 [==============================] - 0s 783us/step - loss: 0.528

In [95]:
#
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [96]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch')

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
324/380 [========================>.....] - ETA: 0s - loss: 0.4336 - accuracy: 0.7976
Epoch 00001: saving model to checkpoints/weights.01.hdf5
380/380 [==============================] - 1s 865us/step - loss: 0.4369 - accuracy: 0.7960
Epoch 2/100
379/380 [============================>.] - ETA: 0s - loss: 0.4345 - accuracy: 0.7943
Epoch 00002: saving model to checkpoints/weights.02.hdf5
380/380 [==============================] - 0s 821us/step - loss: 0.4344 - accuracy: 0.7944
Epoch 3/100
376/380 [============================>.] - ETA: 0s - loss: 0.4338 - accuracy: 0.7949
Epoch 00003: saving model to checkpoints/weights.03.hdf5
380/380 [==============================] - 0s 827us/step - loss: 0.4340 - accuracy: 0.7945
Epoch 4/100
379/380 [============================>.] - ETA: 0s - loss: 0.4318 - accuracy: 0.7949
Epoch 00004: saving model to checkpoints/weights.04.hdf5
380/380 [==============================] - 0s 820us/step - loss: 0.4319 - accuracy: 0.7948
Epoch 5/100
374/380 

328/380 [========================>.....] - ETA: 0s - loss: 0.4115 - accuracy: 0.8087
Epoch 00034: saving model to checkpoints/weights.34.hdf5
380/380 [==============================] - 0s 789us/step - loss: 0.4132 - accuracy: 0.8076
Epoch 35/100
380/380 [==============================] - ETA: 0s - loss: 0.4131 - accuracy: 0.8073
Epoch 00035: saving model to checkpoints/weights.35.hdf5
380/380 [==============================] - 0s 817us/step - loss: 0.4131 - accuracy: 0.8073
Epoch 36/100
379/380 [============================>.] - ETA: 0s - loss: 0.4121 - accuracy: 0.8051
Epoch 00036: saving model to checkpoints/weights.36.hdf5
380/380 [==============================] - 0s 820us/step - loss: 0.4119 - accuracy: 0.8052
Epoch 37/100
375/380 [============================>.] - ETA: 0s - loss: 0.4101 - accuracy: 0.8097
Epoch 00037: saving model to checkpoints/weights.37.hdf5
380/380 [==============================] - 0s 828us/step - loss: 0.4111 - accuracy: 0.8095
Epoch 38/100
379/380 [=======

328/380 [========================>.....] - ETA: 0s - loss: 0.3975 - accuracy: 0.8168
Epoch 00067: saving model to checkpoints/weights.67.hdf5
380/380 [==============================] - 0s 793us/step - loss: 0.3984 - accuracy: 0.8155
Epoch 68/100
314/380 [=======================>......] - ETA: 0s - loss: 0.3959 - accuracy: 0.8162
Epoch 00068: saving model to checkpoints/weights.68.hdf5
380/380 [==============================] - 0s 814us/step - loss: 0.3970 - accuracy: 0.8140
Epoch 69/100
379/380 [============================>.] - ETA: 0s - loss: 0.3968 - accuracy: 0.8173
Epoch 00069: saving model to checkpoints/weights.69.hdf5
380/380 [==============================] - 0s 817us/step - loss: 0.3970 - accuracy: 0.8173
Epoch 70/100
377/380 [============================>.] - ETA: 0s - loss: 0.3983 - accuracy: 0.8162
Epoch 00070: saving model to checkpoints/weights.70.hdf5
380/380 [==============================] - 0s 827us/step - loss: 0.3975 - accuracy: 0.8168
Epoch 71/100
329/380 [=======

366/380 [===========================>..] - ETA: 0s - loss: 0.3881 - accuracy: 0.8206
Epoch 00100: saving model to checkpoints/weights.100.hdf5
380/380 [==============================] - 0s 852us/step - loss: 0.3881 - accuracy: 0.8207
127/127 - 0s - loss: 0.9202 - accuracy: 0.6518 - 136ms/epoch - 1ms/step
Loss: 0.9201602935791016, Accuracy: 0.6517658829689026
